In [24]:
import os
from lxml import etree

input_folder = "../svg/drawing"  
output_folder = "../svg/select_drawing"

A4_width = 297
A4_height = 210
A5_width = 210
A5_height = 148

numbers_to_process = [1, 4, 6, 12, 16, 18, 20,21,25,26,27,28,32,37,38,40,42,43,46,47,48,
                     53,58,60,64,66,98,101,110,138,167,165,166,173,202,204,223,264,263,272,306,
                     111,120,125,135,144,214,213,211,222,226,227,228,237,246,255,262,274,285,284,291,290,289,300,
                     305,163,82,84,71,69,77,76,75,73,85,92,95,115,123,124]  
files_to_process = [f"drawing_{i:03}.svg" for i in numbers_to_process]


# Conversion pour px → mm
def to_mm(value):
    if value.endswith("mm"):
        return float(value.replace("mm", ""))
    elif value.endswith("px"):
        return float(value.replace("px", "")) * 0.264583
    elif value.endswith("pt"):
        return float(value.replace("pt", "")) * 0.352778
    else:
        return float(value)

os.makedirs(output_folder, exist_ok=True)

for filename in files_to_process:
    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, filename)

    tree = etree.parse(input_path)
    root = tree.getroot()
    svg_width = to_mm(root.get("width", "0"))
    svg_height = to_mm(root.get("height", "0"))

    viewBox = root.get("viewBox")
    if viewBox:
        vb_x, vb_y, vb_width, vb_height = map(float, viewBox.split())
    else:
        vb_width, vb_height = svg_width, svg_height

    # Calcul du facteur de scale pour tenir dans A5
    scale_factor = min(A5_width / vb_width, A5_height / vb_height)
    scaled_width = vb_width * scale_factor
    scaled_height = vb_height * scale_factor

    translate_x = (A4_width - scaled_width) / 2
    translate_y = (A4_height - scaled_height) / 2

    # Nouveau SVG racine pour la feuille A4
    new_root = etree.Element("svg", nsmap=root.nsmap)
    new_root.attrib["width"] = str(A4_width)
    new_root.attrib["height"] = str(A4_height)
    new_root.attrib["viewBox"] = f"0 0 {A4_width} {A4_height}"

    # Rectangle blanc pour la feuille A4
    rect = etree.Element("rect")
    rect.attrib["x"] = "0"
    rect.attrib["y"] = "0"
    rect.attrib["width"] = str(A4_width)
    rect.attrib["height"] = str(A4_height)
    rect.attrib["fill"] = "white"
    new_root.append(rect)

    g = etree.Element("g")
    g.attrib["transform"] = f"translate({translate_x},{translate_y}) scale({scale_factor})"
    for child in root:
        g.append(child)

    new_root.append(g)
    new_tree = etree.ElementTree(new_root)
    new_tree.write(output_path, pretty_print=True, xml_declaration=True, encoding="UTF-8")

print("Good")


Good
